# Roberta Classifier on SNLI: baseline
Stanford Natural Langauge Ingerence dataset.
https://huggingface.co/datasets/snli

The SNLI corpus (version 1.0) is a collection of 570k human-written English sentence pairs manually labeled for balanced classification with the labels entailment, contradiction, and neutral, supporting the task of natural language inference (NLI), also known as recognizing textual entailment (RTE).

* training examples: 550152
* test examples: 10000

3 labels.

In [1]:
import json
import os
from typing import List

%pip install datasets
%pip install transformers
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


ROOT_DIR = "drive/My Drive/Colab Notebooks/nlp/results/snli_baseline"
if not os.path.exists(ROOT_DIR):
    os.mkdir(ROOT_DIR)

In [6]:
dataset = load_dataset("snli", split="test")
dataset[1]

Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)


{'hypothesis': 'The church is filled with song.',
 'label': 0,
 'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.'}

In [3]:
def get_datasets(dataset_name, train_size, val_size=1_000, test_size=None, random_seed: int = 42):
    dataset = load_dataset(dataset_name, split="train").filter(lambda d: d['label'] != -1)
    test_dataset = load_dataset(dataset_name, split="test").filter(lambda d: d['label'] != -1)
    # We want test and validation data to be the same for every experiment
    if test_size:
        test_dataset = test_dataset.train_test_split(test_size=test_size, seed=random_seed)["test"]
    train_val_split = dataset.train_test_split(test_size=val_size, seed=random_seed)
    # Validation and test sets
    train_dataset = train_val_split["train"].train_test_split(train_size=train_size, seed=random_seed)["train"]
    val_dataset = train_val_split["test"]
    return train_dataset, val_dataset, test_dataset


class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def __call__(self, examples: List[dict]):
        labels = [example['label'] for example in examples]
        texts = [example['hypothesis'] + f' {self.tokenizer.sep_token} ' + example['premise'] for example in examples]
        tokenizer_output = self.tokenizer(texts, truncation=True, padding=True)
        return {
            'labels': torch.tensor(labels), 
            'input_ids': torch.tensor(tokenizer_output['input_ids']), 
            'attention_mask': torch.tensor(tokenizer_output['attention_mask'])
            }


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, micro_f1, _ = precision_recall_fscore_support(labels, preds, average='micro', zero_division=0)
    _, _, macro_f1, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'micro_f1': micro_f1,
        'micro_precision': precision,
        'micro_recall': recall,
        'macro_f1': macro_f1
    }

In [4]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base', use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', return_dict=True, num_labels=3)
data_collator = DataCollator(tokenizer)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [5]:
TRAIN_SIZES = [20, 100, 1_000, 10_000, 100_000]
for train_size in TRAIN_SIZES:
    train_dataset, val_dataset, test_dataset = get_datasets("snli", train_size, val_size=7_000)
    print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")
    print(train_dataset[0])
    print(val_dataset[0])
    print(test_dataset[0])
    output_dir = os.path.join(ROOT_DIR, f"train_size_{train_size}")

    num_train_epochs = 6 if train_size <= 10_000 else 3

    # https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments
    training_args = TrainingArguments(
        learning_rate=3e-5,
        weight_decay=0.01,
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=0,  # don't have any intuition for the right value here
        logging_dir=output_dir,
        logging_steps=10,
        load_best_model_at_end=True,
        evaluation_strategy='epoch',
        remove_unused_columns=False,
        no_cuda=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
        
    )

    trainer.train()

    test_result = trainer.evaluate(test_dataset)

    print(test_result)

    with open(os.path.join(output_dir, 'test_result.json'), 'w') as f:
        json.dump(test_result, f, indent=4)

    print()

Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-c5e7cc0489dda538.arrow
Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-677b99ebf191dd17.arrow


Train size: 20, Validation size: 7000, Test size: 9824
{'hypothesis': 'A girl is jumping into the pool.', 'label': 2, 'premise': 'A young boy in a red life jacket is swimming in a pool.'}
{'hypothesis': 'A group of people are riding a roller coaster.', 'label': 0, 'premise': 'A group of people riding a yellow roller coaster.'}
{'hypothesis': 'The church has cracks in the ceiling.', 'label': 1, 'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.'}


Epoch,Training Loss,Validation Loss,Accuracy,Micro F1,Micro Precision,Micro Recall,Macro F1
1,No log,1.104674,0.333000,0.333000,0.333000,0.333000,0.166542
2,No log,1.113841,0.333000,0.333000,0.333000,0.333000,0.166542
3,No log,1.129465,0.333000,0.333000,0.333000,0.333000,0.166542
4,No log,1.148920,0.333000,0.333000,0.333000,0.333000,0.166542
5,1.008167,1.169277,0.333000,0.333000,0.333000,0.333000,0.166542
6,1.008167,1.180557,0.333000,0.333000,0.333000,0.333000,0.166542


{'eval_loss': 1.1052757501602173, 'eval_accuracy': 0.32949918566775244, 'eval_micro_f1': 0.32949918566775244, 'eval_micro_precision': 0.32949918566775244, 'eval_micro_recall': 0.32949918566775244, 'eval_macro_f1': 0.1652247147997856, 'epoch': 6.0, 'total_flos': 4092441133248}



Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-c5e7cc0489dda538.arrow
Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-677b99ebf191dd17.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-48e7475b404be1e0.arrow and /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-0a7ebfe579f236a

...

{'eval_loss': 0.4842863380908966, 'eval_accuracy': 0.8422231270358306, 'eval_micro_f1': 0.8422231270358306, 'eval_micro_precision': 0.8422231270358306, 'eval_micro_recall': 0.8422231270358306, 'eval_macro_f1': 0.8415326057804551, 'epoch': 6.0, 'total_flos': 1972652355842688}



Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-c5e7cc0489dda538.arrow
Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-677b99ebf191dd17.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-48e7475b404be1e0.arrow and /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-0a7ebfe579f236a

Train size: 100000, Validation size: 7000, Test size: 9824
{'hypothesis': 'A man is wearing glasses', 'label': 0, 'premise': 'A man in glasses relaxing with his feet up.'}
{'hypothesis': 'A group of people are riding a roller coaster.', 'label': 0, 'premise': 'A group of people riding a yellow roller coaster.'}
{'hypothesis': 'The church has cracks in the ceiling.', 'label': 1, 'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.'}


Epoch,Training Loss,Validation Loss,Accuracy,Micro F1,Micro Precision,Micro Recall,Macro F1
1,0.441431,0.374956,0.868857,0.868857,0.868857,0.868857,0.869015
2,0.292187,0.379542,0.876429,0.876429,0.876429,0.876429,0.875292


Buffered data was truncated after reaching the output size limit.